<h1> MACHINE LEARNING </h1>

In [5]:
import pandas as pd

df = pd.read_csv("film.csv")
df.drop(columns = ["IdFilm", "Titolo", "Anno"], axis = 1, inplace = True)
df.head(3)

,TitoloOriginale,Durata,Genere,Regia,Poster,Budget,IncassoPrimoWeekendUsa,IncassoUsa,IncassoGlobale,Ricavo,ValutazioneImdb,ValutazioneTmdb,ValutazioneRottenTomatoes,ValutazioneMetacritic,ValutazioneMedia
0,The lord of the rings - The fellowship of the ...,178,NaN,Peter Jackson,D:\Documents\Progetti\FilmDB\Poster\Il signore...,93000000.0,47211490.0,319372078.0,8.881951e+08,7.951951e+08,8.9,8.4,9.5,9.3,9.025
1,The lord of the rings - The two towers,179,NaN,Peter Jackson,D:\Documents\Progetti\FilmDB\Poster\Il signore...,94000000.0,62007528.0,345518923.0,9.382429e+08,8.442429e+08,8.8,8.4,9.5,9.2,8.975
2,The lord of the rings - The return of the king,200,NaN,Peter Jackson,D:\Documents\Progetti\FilmDB\Poster\Il signore...,94000000.0,72629713.0,381878219.0,1.138268e+09,1.044268e+09,9.0,8.5,8.6,9.4,8.875


<h3> RIMOZIONE VALORI NAN </h3>

In [278]:
df.isnull().any().any()

np.True_

In [279]:
righe_nan = df[df.isnull().any(axis = 1)]
film_nan = righe_nan["TITOLO"]

In [280]:
df = df.dropna()

<h2> PREPROCESSING </h2>

In [281]:
df.drop(columns = ["TITOLO"], axis = 1, inplace = True)

<h4> LOG-TRANSFORM </h4>

In [282]:
import numpy as np

df["BUDGET_PREVISTO"] = np.log1p(df["BUDGET_PREVISTO"])
df["INCASSO_LORDO"] = np.log1p(df["INCASSO_LORDO"])

<h4> STANDARD SCALER </h4>

In [283]:
from sklearn.preprocessing import StandardScaler

StandardScaler = StandardScaler()
df[["ANNO", "DURATA", "VALUTAZIONE_MEDIA"]] = StandardScaler.fit_transform(df[["ANNO", "DURATA", "VALUTAZIONE_MEDIA"]])

<h4> LABEL ENCODER </h4>

In [284]:
from sklearn.preprocessing import LabelEncoder

LabelEncoder = LabelEncoder()
df["REGIA"] = LabelEncoder.fit_transform(df["REGIA"])
df["MUSICHE"] = LabelEncoder.fit_transform(df["MUSICHE"])

<h4> GET-DUMMIES </h4>

In [285]:
df_lingua_originale = pd.get_dummies(df["LINGUA_ORIGINALE"], prefix = "LINGUA", prefix_sep = "-")
df_paese_produzione = df["PAESE_PRODUZIONE"].str.get_dummies(sep = ", ")
df_genere = df["GENERE"].str.get_dummies(sep = ", ")

df = pd.concat([df, df_lingua_originale, df_paese_produzione, df_genere], axis = 1)

In [286]:
df = df.loc[:, ~df.columns.duplicated()]
df.drop(columns = ["GENERE", "LINGUA_ORIGINALE", "PAESE_PRODUZIONE"], axis = 1, inplace = True)

<h2> DIVISIONE DATAFRAME </h2>

In [287]:
X = df.drop(columns = ["VALUTAZIONE_PERSONALE"])
y = df["VALUTAZIONE_PERSONALE"]

In [288]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [289]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

modello = LinearRegression()
modello.fit(X_train, y_train)

LinearRegression()

<h3> PREDIZIONI </h3>

In [290]:
y_pred = modello.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse} \n\
R2: {r2}")
print(f"PREDIZIONI: {y_pred[:5]} \n\
VALORI REALI: {y_test[:5].values}")

MSE: 1.3896241320644602 
R2: -0.19795183798660365
PREDIZIONI: [10.17144796  7.22789856  5.46399793  9.44018071  7.69408153] 
VALORI REALI: [8.5 6.  7.  9.  8. ]
